In [1]:
import pandas as pd
import osmnx as ox

In [2]:
trip_df = pd.read_csv('../../data/trip_data_normalized_2019-01-01_2019-12-31.csv')
crash_df = pd.read_csv('../../data/crash_data_normalized_with_node_graph.csv')
weather_df = pd.read_csv('../../data/weather_data_normalized_2031-01-01_2021-10-31.csv')

So our goal here:

1. Get the trip and crash datasets to have comparable sets of columns so that they can be directly concatenated.
1. Append the weather dataset's columns by joining on the date for each event.

In [3]:
time_df = trip_df['starttime'].str.split(' ', 1, expand=True)

trip_df_norm = trip_df.drop(columns=[
    'TRIP_ID',
    'Unnamed: 0',
    'starttime',
    'stoptime',
    'start station id',
    'start station name',
    'start station latitude',
    'tripduration',
    'start station longitude',
    'end station id',
    'end station name',
    'end station latitude',
    'end station longitude',
    'bikeid',
    'usertype',
    'birth year',
    'gender'
])

trip_df_norm['NUMBER OF PERSONS INJURED'] = trip_df_norm['NUMBER OF PERSONS KILLED'] = 0
trip_df_norm['NUMBER OF PEDESTRIANS INJURED'] = trip_df_norm['NUMBER OF PEDESTRIANS KILLED'] = 0
trip_df_norm['NUMBER OF CYCLIST INJURED'] = trip_df_norm['NUMBER OF CYCLIST KILLED'] = 0
trip_df_norm['NUMBER OF MOTORIST INJURED'] = trip_df_norm['NUMBER OF MOTORIST KILLED'] = 0
trip_df_norm['EVENT_DIST_FROM_NODE'] = 0

trip_df_norm['EVENT_DATE'] = time_df[0]
trip_df_norm['EVENT_TIME'] = time_df[1]

trip_df_norm['IS_CRASH'] = False

trip_df_norm

,NODE_ID,NODE_LATITUDE,NODE_LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,EVENT_DIST_FROM_NODE,EVENT_DATE,EVENT_TIME,IS_CRASH
0,42430108,40.724267,-74.004739,0,0,0,0,0,0,0,0,0,2019-10-01,00:00:05.6180,False
1,42424864,40.724413,-74.005930,0,0,0,0,0,0,0,0,0,2019-10-01,00:00:05.6180,False
2,42423456,40.723791,-74.006113,0,0,0,0,0,0,0,0,0,2019-10-01,00:00:05.6180,False
3,4602414021,40.723761,-74.006117,0,0,0,0,0,0,0,0,0,2019-10-01,00:00:05.6180,False
4,42452015,40.723132,-74.006215,0,0,0,0,0,0,0,0,0,2019-10-01,00:00:05.6180,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22590805,42427358,40.720650,-73.981345,0,0,0,0,0,0,0,0,0,2019-11-30,23:59:07.8070,False
22590806,7115850098,40.720147,-73.979672,0,0,0,0,0,0,0,0,0,2019-11-30,23:59:07.8070,False
22590807,42449874,40.719879,-73.978792,0,0,0,0,0,0,0,0,0,2019-11-30,23:59:07.8070,False
22590808,42453613,40.719987,-73.978723,0,0,0,0,0,0,0,0,0,2019-11-30,23:59:07.8070,False


In [4]:
crash_df_norm = crash_df.drop(columns=[
    'Unnamed: 0',
    'Unnamed: 0.1',
    'LATITUDE',
    'LONGITUDE',
    'LOCATION',
    'ZIP CODE',
    'BOROUGH',
    *[f'VEHICLE TYPE CODE {i+1}' for i in range(5)],
    *[f'CONTRIBUTING FACTOR VEHICLE {i+1}' for i in range(5)],
    'ON STREET NAME',
    'CROSS STREET NAME',
    'OFF STREET NAME',
    'COLLISION_ID'
]).rename(columns={
    'CRASH DATE': 'EVENT_DATE',
    'CRASH TIME': 'EVENT_TIME',
    'NODE_DIST_FROM_CRASH_M': 'EVENT_DIST_FROM_NODE'
}).assign(
    EVENT_DATE=lambda df: pd.to_datetime(df['EVENT_DATE']).dt.date.astype(str),
    EVENT_TIME=lambda df: df['EVENT_TIME'] + ':00.0000',
    IS_CRASH=lambda df: True
)

crash_df_norm

,EVENT_DATE,EVENT_TIME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,NODE_ID,EVENT_DIST_FROM_NODE,NODE_LATITUDE,NODE_LONGITUDE,IS_CRASH
0,2021-04-16,18:40:00.0000,1.0,0.0,0,0,1,0,0,0,42437644,2.897338,40.748439,-73.984532,True
1,2021-04-01,15:10:00.0000,0.0,0.0,0,0,0,0,0,0,42442895,22.200879,40.740422,-73.979096,True
2,2021-04-10,11:15:00.0000,1.0,0.0,1,0,0,0,0,0,486867432,7.759233,40.719681,-73.992175,True
3,2021-04-13,17:15:00.0000,1.0,0.0,1,0,0,0,0,0,42434807,12.272091,40.746618,-73.993813,True
4,2021-04-15,12:12:00.0000,1.0,0.0,1,0,0,0,0,0,42454798,1.284833,40.706146,-74.006016,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3340,2019-01-03,10:30:00.0000,1.0,0.0,0,0,1,0,0,0,42453104,3.675021,40.731964,-73.990203,True
3341,2019-01-04,19:00:00.0000,1.0,0.0,0,0,1,0,0,0,42442255,1.395292,40.722669,-73.997975,True
3342,2019-01-03,14:30:00.0000,1.0,0.0,1,0,0,0,0,0,42429754,7.054891,40.719907,-73.992918,True
3343,2019-01-03,0:27:00.0000,0.0,0.0,0,0,0,0,0,0,42436142,1.343927,40.735732,-74.006801,True


In [5]:
all_event_df = pd.concat((trip_df_norm, crash_df_norm)).reset_index().drop(columns=['index'])

In [6]:
all_event_df

,NODE_ID,NODE_LATITUDE,NODE_LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,EVENT_DIST_FROM_NODE,EVENT_DATE,EVENT_TIME,IS_CRASH
0,42430108,40.724267,-74.004739,0.0,0.0,0,0,0,0,0,0,0.000000,2019-10-01,00:00:05.6180,False
1,42424864,40.724413,-74.005930,0.0,0.0,0,0,0,0,0,0,0.000000,2019-10-01,00:00:05.6180,False
2,42423456,40.723791,-74.006113,0.0,0.0,0,0,0,0,0,0,0.000000,2019-10-01,00:00:05.6180,False
3,4602414021,40.723761,-74.006117,0.0,0.0,0,0,0,0,0,0,0.000000,2019-10-01,00:00:05.6180,False
4,42452015,40.723132,-74.006215,0.0,0.0,0,0,0,0,0,0,0.000000,2019-10-01,00:00:05.6180,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22594150,42453104,40.731964,-73.990203,1.0,0.0,0,0,1,0,0,0,3.675021,2019-01-03,10:30:00.0000,True
22594151,42442255,40.722669,-73.997975,1.0,0.0,0,0,1,0,0,0,1.395292,2019-01-04,19:00:00.0000,True
22594152,42429754,40.719907,-73.992918,1.0,0.0,1,0,0,0,0,0,7.054891,2019-01-03,14:30:00.0000,True
22594153,42436142,40.735732,-74.006801,0.0,0.0,0,0,0,0,0,0,1.343927,2019-01-03,0:27:00.0000,True


In [7]:
weather_df

,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,...,WT09,WT10,WT11,WT13,WT14,WT15,WT16,WT18,WT19,WT22
0,2013-01-01,11.23875,1843.500,0.000741,0.0,0.722222,39.785714,27.571429,33.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,2013-01-02,9.70375,1201.750,0.000000,0.0,0.350000,35.500000,22.285714,24.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-01-03,7.40750,941.250,0.000000,0.0,0.287500,32.714286,22.642857,25.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-01-04,11.32250,1220.750,0.000000,0.0,0.273333,37.000000,27.071429,33.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-01-05,8.02625,1126.500,0.000000,0.0,0.178571,41.714286,29.714286,33.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3221,2021-10-27,13.67375,848.500,1.602989,0.0,0.000000,62.923077,52.769231,55.800000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3222,2021-10-28,6.34750,1180.500,0.008101,0.0,0.000000,59.615385,46.769231,47.200000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3223,2021-10-29,13.67250,1912.750,0.043947,0.0,0.000000,57.076923,45.769231,47.200000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3224,2021-10-30,8.47125,119.750,0.648353,0.0,0.000000,62.461538,51.615385,54.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
all_event_df_with_weather = pd.merge(all_event_df, weather_df, left_on='EVENT_DATE', right_on='DATE', how='left')

Looks like some of these crashes happened in 2012. We'll drop them:

In [9]:
all_event_df_with_weather.query('DATE.isna()')['EVENT_DATE']

22592375    2021-11-01
22592377    2021-11-01
Name: EVENT_DATE, dtype: object

In [10]:
all_event_df_with_weather.dropna(inplace=True)

In [13]:
all_event_df_with_weather['IS_CRASH'].value_counts()

False    22590810
True         3343
Name: IS_CRASH, dtype: int64

In [11]:
all_event_df_with_weather.to_csv('../../data/unified_dataset.csv')